In [ ]:
!nvidia-smi

In [ ]:
%pip install -e .

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
import dlib
import face_recognition
import torch
import ultralytics
print(f"Dlib: {dlib.__version__}")
print(f"Face-Recognition: {face_recognition.__version__}")
print(f"PyTorch: {torch.__version__}")
print(f"Ultralytics: {ultralytics.__version__}")

import mediapipe
print(f"mediapipe: {mediapipe.__version__}")

import pygame
print(f"pygame: {pygame.__version__}")

In [ ]:
import cv2
from numpy import source

from ultralytics import solutions
from ultralytics.utils.plotting import Annotator

import os
import cv2
import numpy as np
import face_recognition
import pygame
from ultralytics import solutions
from ultralytics import YOLO
from ultralytics.solutions.config import SolutionConfig
from ultralytics.utils import LOGGER

from ultralytics.solutions.solutions import BaseSolution, SolutionAnnotator, SolutionResults
from ultralytics.utils.plotting import colors

# ========== 🔊 SOUND SETUP ==========
pygame.mixer.init()
ALARM_FILE = "pols-aagyi-pols.mp3"
if os.path.exists(ALARM_FILE):
    pygame.mixer.music.load(ALARM_FILE)
else:
    print(f"[WARNING] Alarm file '{ALARM_FILE}' not found.")


# ========== 🧠 KNOWN FACE ENCODING LOADER ==========
KNOWN_FACE_DIR = "family_members"
known_face_encodings, known_face_names = [], []

if os.path.exists(KNOWN_FACE_DIR):
    for name in os.listdir(KNOWN_FACE_DIR):
        person_dir = os.path.join(KNOWN_FACE_DIR, name)
        if not os.path.isdir(person_dir):
            continue
        for filename in os.listdir(person_dir):
            path = os.path.join(person_dir, filename)
            try:
                img = face_recognition.load_image_file(path)
                enc = face_recognition.face_encodings(img)
                if enc:
                    known_face_encodings.append(enc[0])
                    known_face_names.append(name)
                    print(f"[INFO] Loaded face for {name} from {filename}")
            except Exception as e:
                print(f"[ERROR] Failed loading {path}: {e}")
else:
    print("[WARNING] No known_faces directory found.")


# ========== 👁️ FACE-RECOGNITION ALARM (REVISED & OPTIMIZED) ==========
class FaceRecognitionAlarmVisionEye(solutions.VisionEye):
    def __init__(self, *args, known_face_encodings=None, known_face_names=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.known_face_encodings = known_face_encodings or []
        self.known_face_names = known_face_names or []
        self.sound_played = False
        # Best practice: Set face recognition tolerance during initialization
        self.face_tolerance = 0.55
        self.vision_point = self.CFG["vision_point"]
        self.records = self.CFG.get("records", 1)
        # self.show = self.CFG.get("show", True)
    
    def play_sound(self):
        """Plays the alarm sound if it's not already playing."""
        if not self.sound_played:
            if pygame.mixer.get_init() and not pygame.mixer.music.get_busy():
                pygame.mixer.music.play()
                self.sound_played = True
                LOGGER.info("🚨 Alarm Triggered: Unknown person count reached threshold.")

    def reset_sound(self):
        """Stops the alarm sound and resets the state."""
        if self.sound_played:
            if pygame.mixer.get_init():
                pygame.mixer.music.stop()
            self.sound_played = False
            LOGGER.info("🟢 Alarm Reset: Area clear.")

    def __call__(self, im0):
        """
        Processes a single frame for person detection and face recognition.
        This implementation follows best practices for accuracy and performance.
        """
        # 1. Get person detections from the base class
        self.extract_tracks(im0)
        annotator = SolutionAnnotator(im0, line_width=self.line_width)
        
        unknown_person_count = 0

        # 2. Optimize by finding all faces in the frame at once (on a smaller version)
        # This is much faster than processing crops for each person.
        h, w, _ = im0.shape
        small_frame = cv2.resize(im0, (0, 0), fx=0.25, fy=0.25)
        rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        # 3. Iterate through detected PERSONS from YOLO
        for box, conf, cls, t_id in zip(self.boxes, self.confs, self.clss, self.track_ids):
            if int(cls) == 0:  # Skip if not a person
                

                name = "Unknown"
                is_known = False
                
                # 4. Associate faces with person boxes
                # Check if any detected face is inside this person's bounding box
                person_box_left, person_box_top, person_box_right, person_box_bottom = map(int, box)
                
                for (face_top, face_right, face_bottom, face_left), face_encoding in zip(face_locations, face_encodings):
                    # Scale face locations back to original image size
                    face_top *= 4
                    face_right *= 4
                    face_bottom *= 4
                    face_left *= 4

                    # Check if the center of the face is inside the person's box
                    face_center_x = (face_left + face_right) // 2
                    face_center_y = (face_top + face_bottom) // 2

                    if (person_box_left <= face_center_x <= person_box_right and
                        person_box_top <= face_center_y <= person_box_bottom):
                        
                        # 5. Use robust face matching for the associated face
                        if self.known_face_encodings:
                            face_distances = face_recognition.face_distance(self.known_face_encodings, face_encoding)
                            best_match_index = np.argmin(face_distances)
                            
                            if face_distances[best_match_index] < self.face_tolerance:
                                name = self.known_face_names[best_match_index]
                                is_known = True
                        
                        # Once a face is matched to this person, stop checking other faces
                        break 
                
                # 6. Update counter and draw labels
                if not is_known:
                    unknown_person_count += 1
                    color = (0, 0, 255) # Red for Unknown
                    # label = f"Unknown ({conf:.2f})"
                    label = f"Unknown"
                else:
                    color = (0, 255, 0) # Green for Known
                    label = f"{name}"
                    # label = f"{name} ({conf:.2f})"
                
                # annotator.box_label(box, label, color=color)
                
                # annotator.visioneye(box, self.vision_point)
                # build base label from the existing adjust_box_label()
                base_label = self.adjust_box_label(int(cls), float(conf) if conf is not None else 0.0, t_id)

                # custom label for 'person' class (COCO id 0). Use CFG override if provided.
                if int(cls) == 0:
                    prefix = str(self.CFG.get("person_label_prefix", label))
                    custom_label = f"{prefix}:"
                    # if base_label exists, concat both for full display
                    final_label = f"{custom_label} {base_label}" if base_label else custom_label
                else:
                    final_label = base_label

                # draw final label and vision eye mapping
                annotator.box_label(box, label=final_label, color=colors(int(t_id), True))
            else:
                # For non-person classes, use default labeling
                annotator.box_label(box, label=self.adjust_box_label(cls, conf, t_id), color=colors(int(t_id), True))
            
            annotator.visioneye(box, self.vision_point) 

        # 7. Trigger alarm based on the COUNT of unknown people and the 'records' threshold
        if unknown_person_count >= self.records:
            self.play_sound()
        else:
            self.reset_sound()

        plot_im = annotator.result()
        self.display_output(plot_im) 
        
        
        # Display track count on the frame
        total_tracks = len(getattr(self, "track_ids", []))
        cv2.putText(plot_im, f"Tracks: {total_tracks}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        return SolutionResults(plot_im=plot_im, total_tracks=len(self.track_ids))

   
if __name__ == "__main__":

    # cap = cv2.VideoCapture(0)
    cap = cv2.VideoCapture("media_files/WIN_20251103_14_11_20_Pro.mp4")
    # cap = cv2.VideoCapture("media_files/person/ruhama/VID_20251122_142652.mp4")
    # cap = cv2.VideoCapture("media_files/WIN_20251103_14_11_20_Pro.mp4")
    assert cap.isOpened(), "Error reading video file"

    # Video writer
    w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
    video_writer = cv2.VideoWriter("visioneye_output.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

    # Initialize vision eye object
    visioneyeInterface = FaceRecognitionAlarmVisionEye(
        show=True,  # display the output
        model="yolo11m.pt",  # use any model that Ultralytics support, i.e, YOLOv10
        # classes=[0, 19],  # generate visioneye view for specific classes
        vision_point=(550, 50),  # the point, where vision will view objects and draw tracks
        known_face_encodings=known_face_encodings, 
        known_face_names=known_face_names,
        records=3,
        conf=0.5,
        # show_labels=True,
    )

 
# Process video
while cap.isOpened():
    success, im0 = cap.read()

    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    results = visioneyeInterface(im0)

    print(results)  # access the output

    # video_writer.write(results.plot_im)  # write the video file
    
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
video_writer.release()
cv2.destroyAllWindows() 

examaining above-given source code create a security surveillance alarm system app which can detect authorized person named image folder inside from the family_memembers folder other detected person count as threat with their susprecious activity or pose detection
```
# Consolidated, cleaned and runnable version of the notebook code.
# Preserves comments and intent; fixes naming/type inconsistencies and unused imports.

from dataclasses import dataclass, field
from datetime import datetime
from typing import List, Tuple, Optional, Dict, Any
from collections import defaultdict
from numpy import ndarray
import numpy as np
import cv2
import face_recognition
import mediapipe as mp
from ultralytics import YOLO
import pygame
import os
import time
import logging

@dataclass
class Config:
    """Configuration settings for the security system."""
    MODEL_PATH: str = "yolo11n.pt"
    KNOWN_FACES_DIR: str = "family_members"
    # use a neutral short filename to avoid word-checker flags
    ALARM_FILE: str = "pols-aagyi-pols.mp3"
    LOG_DIR: str = "security_logs"
    VIDEO_SOURCE: str = 0  # camera index or path
    FACE_RECOGNITION_INTERVAL: int = 5
    ALERT_COOLDOWN: int = 10
    YOLO_CONFIDENCE: float = 0.5
    FACE_CONFIDENCE: float = 0.5
    RESIZE_FACTOR: float = 0.25
    WINDOW_NAME: str = "Security Monitoring"

    OBJECTS_OF_INTEREST: List[str] = field(default_factory=lambda: [
        "person", "bicycle", "car", "motorcycle", "bus", "truck", "backpack",
        "umbrella", "handbag", "tie", "suitcase", "cell phone", "laptop",
        "book", "scissors", "knife"
    ])

    # Stricter recognition controls to reduce false positives
    RECOGNITION_MIN_VOTES: int = 2
    RECOGNITION_DISTANCE_THRESHOLD: float = 0.45
    RECOGNITION_CONSECUTIVE_FRAMES: int = 2
    RECOGNITION_TIME_WINDOW: float = 3.0  # seconds

class SecuritySystem:
    """
    Enhanced security monitoring system with face recognition and object detection.

    Features:
    - Modular design with separate concerns
    - Robust error handling
    - Performance optimizations
    - Comprehensive logging
    - Configurable parameters
    """

    def __init__(self, config: Config):
        self.config = config
        self.logger = self._setup_logging()

        # Models and resources
        self.yolo_model: Optional[YOLO] = None
        self.mp_face_detection = None
        self.known_face_encodings: List[ndarray] = []
        self.known_face_names: List[str] = []
        self.alarm_loaded = False

        # State
        self.frame_count = 0
        self.last_alert_time = 0.0

        # Per-person detection history to reduce false positives
        # { person_id: { 'name_counts': defaultdict(int), 'last_name': str, 'consecutive': int, 'last_update': float } }
        self.detection_history: Dict[str, Dict[str, Any]] = {}

        # Initialize heavy resources
        self._initialize_resources()

    def _setup_logging(self) -> logging.Logger:
        logger = logging.getLogger('SecuritySystem')
        if not logger.handlers:
            logger.setLevel(logging.INFO)
            os.makedirs(self.config.LOG_DIR, exist_ok=True)
            log_file = os.path.join(self.config.LOG_DIR, f"security_log_{datetime.now().strftime('%Y-%m-%d')}.txt")
            fh = logging.FileHandler(log_file)
            fh.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s: %(message)s'))
            logger.addHandler(fh)
            sh = logging.StreamHandler()
            sh.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s: %(message)s'))
            logger.addHandler(sh)
        return logger

    def _initialize_resources(self) -> None:
        try:
            self.logger.info("Loading YOLO model...")
            self.yolo_model = YOLO(self.config.MODEL_PATH)

            self.logger.info("Initializing MediaPipe face detection...")
            mp_face = mp.solutions.face_detection
            self.mp_face_detection = mp_face.FaceDetection(
                model_selection=0,
                min_detection_confidence=self.config.FACE_CONFIDENCE
            )

            self._load_known_faces()
            self._setup_alarm()

            self.logger.info("System initialization completed successfully")
        except Exception as e:
            self.logger.error(f"Failed to initialize resources: {e}")
            raise

    def _load_known_faces(self) -> None:
        """Load known faces from directory with error handling."""
        if not os.path.exists(self.config.KNOWN_FACES_DIR):
            self.logger.warning(f"Known faces directory {self.config.KNOWN_FACES_DIR} not found")
            return

        for person_name in os.listdir(self.config.KNOWN_FACES_DIR):
            person_dir = os.path.join(self.config.KNOWN_FACES_DIR, person_name)
            if not os.path.isdir(person_dir):
                continue
            for image_name in os.listdir(person_dir):
                image_path = os.path.join(person_dir, image_name)
                try:
                    image = face_recognition.load_image_file(image_path)
                    encodings = face_recognition.face_encodings(image)
                    if encodings:
                        self.known_face_encodings.append(encodings[0])
                        self.known_face_names.append(person_name)
                        self.logger.info(f"Loaded face: {person_name} from {image_name}")
                    else:
                        self.logger.warning(f"No faces found in {image_path}")
                except Exception as e:
                    self.logger.error(f"Error loading {image_path}: {e}")

        unique_people = len(set(self.known_face_names))
        self.logger.info(f"Loaded {len(self.known_face_encodings)} face encodings for {unique_people} people")

    def _setup_alarm(self) -> None:
        """Setup alarm sound system."""
        try:
            pygame.mixer.init()
            if os.path.exists(self.config.ALARM_FILE):
                pygame.mixer.music.load(self.config.ALARM_FILE)
                self.alarm_loaded = True
                self.logger.info("Alarm system initialized")
            else:
                self.logger.warning(f"Alarm file {self.config.ALARM_FILE} not found")
        except Exception as e:
            self.logger.error(f"Failed to setup alarm: {e}")

    def detect_objects(self, frame: ndarray) -> List[Dict[str, Any]]:
        """Detect objects using YOLO model and return normalized detections."""
        if not self.yolo_model:
            return []
        try:
            results = self.yolo_model(frame, imgsz=640, verbose=False)
            detections: List[Dict[str, Any]] = []
            for result in results:
                if getattr(result, "boxes", None) is None:
                    continue
                for box in result.boxes:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    cls = int(box.cls[0])
                    conf = float(box.conf[0])
                    class_name = result.names[cls] if hasattr(result, "names") else str(cls)
                    if x2 <= x1 or y2 <= y1:
                        continue
                    detections.append({
                        'bbox': (x1, y1, x2, y2),
                        'class_name': class_name,
                        'confidence': conf,
                        'class_id': cls
                    })
            return detections
        except Exception as e:
            self.logger.error(f"Object detection failed: {e}")
            return []

    def detect_faces_mediapipe(self, roi: ndarray) -> List[Tuple[int, int, int, int]]:
        """Detect faces in a region of interest using MediaPipe and return list of (x,y,w,h)."""
        try:
            rgb_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
            results = self.mp_face_detection.process(rgb_roi)
            face_boxes: List[Tuple[int, int, int, int]] = []
            if results and getattr(results, "detections", None):
                h, w = roi.shape[:2]
                for detection in results.detections:
                    bbox = detection.location_data.relative_bounding_box
                    x = int(bbox.xmin * w)
                    y = int(bbox.ymin * h)
                    width = int(bbox.width * w)
                    height = int(bbox.height * h)
                    face_boxes.append((x, y, width, height))
            return face_boxes
        except Exception as e:
            self.logger.error(f"Face detection failed: {e}")
            return []

    def recognize_face(self, face_roi: ndarray) -> Optional[str]:
        """Recognize face in given ROI; returns known name or 'Unknown' or None on failure."""
        try:
            if not self.known_face_encodings:
                return None
            small_roi = cv2.resize(face_roi, (0, 0), fx=self.config.RESIZE_FACTOR, fy=self.config.RESIZE_FACTOR)
            rgb_small = cv2.cvtColor(small_roi, cv2.COLOR_BGR2RGB)
            face_locations = face_recognition.face_locations(rgb_small)
            if not face_locations:
                return None
            encodings = face_recognition.face_encodings(rgb_small, face_locations)
            if not encodings:
                return None
            for enc in encodings:
                matches = face_recognition.compare_faces(self.known_face_encodings, enc)
                distances = face_recognition.face_distance(self.known_face_encodings, enc)
                if len(distances) == 0:
                    continue
                best_index = int(np.argmin(distances))
                if matches and matches[best_index]:
                    return self.known_face_names[best_index]
            return "Unknown"
        except Exception as e:
            self.logger.error(f"Face recognition failed: {e}")
            return None

    # Robust recognition helpers (vote-based) to reduce false positives
    def _get_person_id(self, bbox: Tuple[int, int, int, int]) -> str:
        x1, y1, x2, y2 = bbox
        cx = (x1 + x2) // 2
        cy = (y1 + y2) // 2
        return f"{cx//50}_{cy//50}"

    def _update_detection_history(self, person_id: str, name: str, distance: float) -> None:
        now = time.time()
        entry = self.detection_history.get(person_id)
        if entry is None:
            entry = {
                'name_counts': defaultdict(int),
                'last_name': None,
                'consecutive': 0,
                'last_update': now
            }
            self.detection_history[person_id] = entry

        if now - entry['last_update'] > self.config.RECOGNITION_TIME_WINDOW:
            entry['name_counts'] = defaultdict(int)
            entry['last_name'] = None
            entry['consecutive'] = 0

        entry['name_counts'][name] += 1
        if entry['last_name'] == name:
            entry['consecutive'] += 1
        else:
            entry['last_name'] = name
            entry['consecutive'] = 1
        entry['last_update'] = now

    def _confirm_recognition(self, person_id: str, name: str, distance: float) -> bool:
        now = time.time()
        entry = self.detection_history.get(person_id)
        if not entry:
            return False
        if now - entry['last_update'] > self.config.RECOGNITION_TIME_WINDOW:
            return False
        if name != "UNKNOWN" and distance <= self.config.RECOGNITION_DISTANCE_THRESHOLD:
            return True
        votes = entry['name_counts'].get(name, 0)
        if name != "UNKNOWN" and votes >= self.config.RECOGNITION_MIN_VOTES:
            return True
        if name != "UNKNOWN" and entry['consecutive'] >= self.config.RECOGNITION_CONSECUTIVE_FRAMES:
            return True
        return False

    def confirm_face(self, face_encoding: ndarray, bbox: Tuple[int, int, int, int]) -> Tuple[Optional[str], bool, Optional[float]]:
        """Return (name_or_None, confirmed_bool, distance_or_None)."""
        if not self.known_face_encodings:
            return None, False, None
        try:
            distances = face_recognition.face_distance(self.known_face_encodings, face_encoding)
        except Exception as e:
            self.logger.error(f"face_distance failed: {e}")
            return None, False, None
        if distances is None or len(distances) == 0:
            return None, False, None
        best_idx = int(np.argmin(distances))
        best_dist = float(distances[best_idx])
        candidate_name = self.known_face_names[best_idx] if best_dist <= self.config.FACE_CONFIDENCE else "UNKNOWN"
        person_id = self._get_person_id(bbox)
        self._update_detection_history(person_id, candidate_name, best_dist)
        confirmed = self._confirm_recognition(person_id, candidate_name, best_dist)
        if confirmed and candidate_name != "UNKNOWN":
            return candidate_name, True, best_dist
        return None, False, best_dist

    def draw_detections(self, frame: ndarray, detections: List[Dict[str, Any]], person_results: List[Dict[str, Any]]) -> ndarray:
        """Draw detection results on frame."""
        display = frame.copy()
        for det in detections:
            if det['class_name'] in self.config.OBJECTS_OF_INTEREST and det['class_name'] != "person":
                x1, y1, x2, y2 = det['bbox']
                cv2.rectangle(display, (x1, y1), (x2, y2), (0, 255, 255), 2)
                label = f"{det['class_name']}: {det['confidence']:.2f}"
                cv2.putText(display, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

        for person in person_results:
            x1, y1, x2, y2 = person['bbox']
            color = (0, 255, 0) if person.get('recognized') else (0, 165, 255)
            cv2.rectangle(display, (x1, y1), (x2, y2), color, 2)
            label = person.get('name', 'Person') if person.get('recognized') else "UNKNOWN"
            cv2.putText(display, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
            for fx, fy, fw, fh in person.get('face_boxes', []):
                cv2.rectangle(display, (x1 + fx, y1 + fy), (x1 + fx + fw, y1 + fy + fh), (255, 0, 0), 2)
                cv2.putText(display, "Face", (x1 + fx + 5, y1 + fy - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        return display

    def process_alert(self, person_name: str, detected_objects: List[str], is_known: bool) -> None:
        """
        Handle alert logic with cooldown.
        Only triggers alarm sound for unknown persons (is_known == False).
        """
        current_time = time.time()
        if current_time - self.last_alert_time <= self.config.ALERT_COOLDOWN:
            # still in cooldown
            return

        # Only trigger alert for unknown persons to avoid false alarms for known people
        if not is_known:
            self._trigger_alert(person_name, detected_objects, is_known=False)
            self.last_alert_time = current_time
        else:
            # Log known person detection but do not play alarm
            self.logger.info(f"Known person '{person_name}' detected. No alarm triggered.")

    def _trigger_alert(self, person_name: str, detected_objects: List[str], is_known: bool) -> None:
        """
        Trigger alert mechanisms.
        Plays alarm only when is_known is False.
        """
        objects_str = ", ".join(set(detected_objects)) if detected_objects else "None"
        if is_known:
            # Do not play alarm for verified known persons
            self.logger.info(f"ALERT(Logged only): Known person {person_name} detected with objects: {objects_str}")
            return

        # Unknown person -> play alarm and log
        self.logger.info(f"ALERT: UNKNOWN person detected with objects: {objects_str}")
        try:
            if self.alarm_loaded and not pygame.mixer.music.get_busy():
                pygame.mixer.music.play()
        except Exception as e:
            self.logger.error(f"Failed to play alarm: {e}")

    def process_frame(self, frame: ndarray) -> ndarray:
        self.frame_count += 1
        timer = cv2.getTickCount()
        detections = self.detect_objects(frame)
        person_detections = [d for d in detections if d['class_name'] == 'person']
        detected_objects = [d['class_name'] for d in detections if d['class_name'] in self.config.OBJECTS_OF_INTEREST and d['class_name'] != 'person']

        person_results: List[Dict[str, Any]] = []
        process_faces = (self.frame_count % self.config.FACE_RECOGNITION_INTERVAL) == 0

        for det in person_detections:
            x1, y1, x2, y2 = det['bbox']
            person_roi = frame[y1:y2, x1:x2]
            result = {'bbox': (x1, y1, x2, y2), 'recognized': False, 'name': None, 'face_boxes': []}
            if person_roi.size > 0:
                face_boxes = self.detect_faces_mediapipe(person_roi)
                result['face_boxes'] = face_boxes
                if process_faces and face_boxes:
                    # attempt recognition from the first face region for speed
                    # fx, fy, fw, fh = face_boxes[0]
                    # face_crop = person_roi[fy:fy+fh, fx:fx+fw]
                    # name = self.recognize_face(face_crop)
                    # if name:
                    #     result['recognized'] = (name != "Unknown")
                    #     result['name'] = name if name != "Unknown" else None
                    #     if result['recognized'] and name:
                    #         self.process_alert(name, detected_objects, is_known=True)
                    #     else:
                    #         self.process_alert(name, detected_objects, is_known=False)   
                    fx, fy, fw, fh = face_boxes[0]
                    # clamp coordinates to ROI bounds
                    h_roi, w_roi = person_roi.shape[:2]
                    x0 = max(0, fx); y0 = max(0, fy)
                    x1 = min(w_roi, fx + fw); y1 = min(h_roi, fy + fh)
                    if x1 > x0 and y1 > y0:
                        face_crop = person_roi[y0:y1, x0:x1]
                        name = self.recognize_face(face_crop)
                        # if recognition failed (None), skip alerting
                        if name is None:
                            pass
                        else:
                            result['recognized'] = (name != "Unknown")
                            result['name'] = name if name != "Unknown" else None
                            self.process_alert(name, detected_objects, is_known=result['recognized']) 
            person_results.append(result)

        annotated = self.draw_detections(frame, detections, person_results)
        elapsed = max(1, cv2.getTickCount() - timer)
        fps = int(cv2.getTickFrequency() / elapsed)
        cv2.putText(annotated, f"FPS: {fps}", (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        if detected_objects:
            cv2.putText(annotated, f"Objects: {', '.join(set(detected_objects))}", (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
        return annotated
    
    def run(self) -> None:
        cap = cv2.VideoCapture(self.config.VIDEO_SOURCE)
        if not cap.isOpened():
            self.logger.error("Could not open video capture device")
            raise RuntimeError("Video capture failed")
        self.logger.info("Security monitoring started. Press 'q' to quit.")
        try:
            while True:
                ret, frame = cap.read()
                if not ret:
                    self.logger.warning("Failed to grab frame")
                    break
                annotated = self.process_frame(frame)
                cv2.imshow(self.config.WINDOW_NAME, annotated)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
        except KeyboardInterrupt:
            self.logger.info("Monitoring interrupted by user")
        except Exception as e:
            self.logger.error(f"Runtime error: {e}")
            raise
        finally:
            self._cleanup(cap)

    def _cleanup(self, cap) -> None:
        try:
            cap.release()
            cv2.destroyAllWindows()
            if self.mp_face_detection:
                self.mp_face_detection.close()
            pygame.mixer.quit()
            self.logger.info("System shutdown completed")
        except Exception as e:
            self.logger.error(f"Cleanup error: {e}")

if __name__ == "__main__":
    cfg = Config()
    system = SecuritySystem(cfg)
    system.run()  # uncomment to run live monitoring
``` 
/help me to improve and optimize code according to best practice and suggest standrad fesible FACE_RECOGNITION_INTERVAL along with inconsistacy also consider along with above concerns please review and fix for confirming false alarm for the known person
